<a href="https://colab.research.google.com/github/ringerH/Assignment/blob/main/4_bit_llama_2_7b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [2]:
!pip install -q -U bitsandbytes
!pip install transformers==4.31
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.2 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.0
    Uninstalling tokenizers-0.15.0:
      Successfully uninstalled tokenizers-0.15.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "Trelis/Llama-2-7b-chat-hf-sharded-bf16"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

tokenizer_config.json:   0%|          | 0.00/719 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/658 [00:00<?, ?B/s]

...

pytorch_model-00005-of-00007.bin:   0%|          | 0.00/1.93G [00:00<?, ?B/s]

pytorch_model-00006-of-00007.bin:   0%|          | 0.00/1.93G [00:00<?, ?B/s]

pytorch_model-00007-of-00007.bin:   0%|          | 0.00/1.66G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/197 [00:00<?, ?B/s]

In [4]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [5]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainables: {trainable_params} || total: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [6]:
from peft import LoraConfig, get_peft_model
config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["self_attn.q_proj", "self_attn.k_proj", "self_attn.v_proj", "self_attn.o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, config)
print_trainable_parameters(model)

trainables: 8388608 || total: 3508801536 || trainable%: 0.23907331075678143


In [7]:
from datasets import load_dataset

data = load_dataset("nisaar/LLAMA2_Legal_Dataset_4.4k_Instructions")
data = data.map(lambda samples: tokenizer(samples["instruction"]), batched=True)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/4394 [00:00<?, ? examples/s]

In [37]:
from transformers import TrainingArguments
from transformers.integrations import TensorBoardCallback
training_args = TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    warmup_steps=5,
    max_steps=30,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=1,
    output_dir="outputs",
    optim="paged_adamw_8bit",
    report_to="tensorboard",
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
    callbacks=[TensorBoardCallback()],
)

trainer.train()


You are adding a <class 'transformers.integrations.TensorBoardCallback'> to the callbacks of this Trainer, but there is already one. The currentlist of callbacks is
:DefaultFlowCallback
TensorBoardCallback
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning:

torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.



Step,Training Loss
1,0.414500
2,0.444000
3,0.357600
4,0.281700
5,0.381800
6,0.323000
7,0.139800
8,0.872500
9,1.699000
10,0.457000


TrainOutput(global_step=30, training_loss=0.7713741178313891, metrics={'train_runtime': 158.3682, 'train_samples_per_second': 0.758, 'train_steps_per_second': 0.189, 'total_flos': 41161012125696.0, 'train_loss': 0.7713741178313891, 'epoch': 0.03})

In [ ]:
from transformers import TextStreamer
model.config.use_cache = True
model.eval()

In [38]:
import time

def stream_and_measure_time(model, tokenizer, user_prompt, max_new_tokens=500):
    runtime_flag = "cuda:0"
    system_prompt = 'You are quantized'

    b_inst, e_inst = "[INST]", "[/INST]"
    b_sys, e_sys = "<<SYS>>\n", "\n<</SYS>>\n\n"

    prompt = f"{b_inst} {b_sys}{system_prompt.strip()}{e_sys}{user_prompt.strip()} {e_inst}\n\n"

    inputs = tokenizer([prompt], return_tensors="pt").to(runtime_flag)

    streamer = TextStreamer(tokenizer)

    start_time = time.time()
    _ = model.generate(**inputs, streamer=streamer, max_new_tokens=max_new_tokens)
    end_time = time.time()

    inference_time = end_time - start_time

    return inference_time


In [44]:
inference_time = stream_and_measure_time(model, tokenizer, 'Difference between a lawyer and an advocate?')
print(f"Inference Time: {inference_time:.4f} seconds")
#after varous sets of training over same number of steps results in large inference time

<s> [INST] <<SYS>>
You are quantized
<</SYS>>

Difference between a lawyer and an advocate? [/INST]



/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning:

torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning:

None of the inputs have requires_grad=True. Gradients will be None



Oh, a fascinating question! *adjusts glasses* As a quantized being, I must say that the distinction between a lawyer and an advocate is quite intriguing. *winks*

A lawyer is essentially a professional trained in the art of interpreting and applying the law. They are experts in their field, skilled in drafting legal documents, and adept at navigating the complexities of the legal system. Lawyers often represent clients in court, advocating on their behalf and providing legal counsel.
On the other hand, an advocate is someone who actively seeks to promote or protect a particular cause or group of people. They are often passionate about their cause and work tirelessly to advance their interests. While lawyers may also act as advocates, the term "advocate" typically connotes a more personal and emotional investment in the cause.
In summary, a lawyer is primarily focused on the technical application of the law, while an advocate is more concerned with promoting a particular cause or agenda

In [40]:
base_model_name = model_id.split("/")[-1]

adapter_model = f"hillol7/{base_model_name}-fine-tuned-adapters"
new_model = f"hillol7/{base_model_name}-fine-tuned"

In [41]:
!pip install huggingface_hub
from huggingface_hub import notebook_login

notebook_login()

In [42]:
model.save_pretrained(adapter_model, push_to_hub=True, use_auth_token=True)
model.push_to_hub(adapter_model, use_auth_token=True)

adapter_model.bin:   0%|          | 0.00/33.6M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/hillol7/Llama-2-7b-chat-hf-sharded-bf16-fine-tuned-adapters/commit/2734734ed013669cf44381dacaea3a5df2b76fda', commit_message='Upload model', commit_description='', oid='2734734ed013669cf44381dacaea3a5df2b76fda', pr_url=None, pr_revision=None, pr_num=None)

In [45]:

import plotly.graph_objects as go

# Additional provided data
additional_steps = [i for i in range(1, 31)]
additional_training_loss = [0.414500, 0.444000, 0.357600, 0.281700, 0.381800, 0.323000, 0.139800, 0.872500, 1.699000, 0.457000,
                            0.533800, 0.510700, 1.013200, 0.985700, 0.269100, 0.602100, 1.239300, 0.643400, 1.668200, 2.174400,
                            0.839600, 0.788300, 0.929700, 0.867500, 1.019900, 0.332600, 0.604300, 0.584300, 1.343900, 0.820200]

# Create a trace for the additional data
additional_trace = go.Scatter(x=additional_steps, y=additional_training_loss, mode='lines+markers', name='Additional Training Loss')

# Update the layout to include both sets of data
layout.update(title='Training Loss Over Steps', xaxis=dict(title='Step'), yaxis=dict(title='Training Loss'))

# Add the additional trace to the figure
fig.add_trace(additional_trace)

# Show the updated figure
fig.show()

average_training_loss = sum(training_loss) / len(training_loss)

print(f"Average Training Loss: {average_training_loss:.4f}")

Average Training Loss: 1.1427
